In [7]:
import numpy as np
from cvxopt import matrix,solvers
import math 
import time
from fractions import Fraction

We have done the production planning application and defined the A,B and C matrix in standard form(max C^T.x such 
that Ax<=b, modify those accordingly for different examples,(for max C^T.x with constraint AX <=B, C array values should be written in negative, if we have to maximize function )

In [8]:
A=np.array([[100,60,120,70],[-2,-1,0,0],[0,0,-3,-2],[1,1,0,0],[0,0,1,1]]).astype(np.int64)
B=np.array([990,-7,-16,10,6]).astype(np.int64)
C=np.array([-40,-20,-30,-10]).astype(np.int64)
A=A+Fraction()
B=B+Fraction()
C=C+Fraction()
solution=[]
iterations=[]

In below blocks, we have done preprocessing to include slack variables, and make the tableau for using simplex algorithms
which will give us cuts, and that cut will be added to our constraints and this process will repeat again till we get
the optimal integer solution

In [9]:
def preprocessing_CP(c,a,b):
    Nc=np.shape(a)[0] #  no of constraints
    c=np.hstack((c,(np.zeros(Nc+1)))).astype(np.int64)
    I=np.eye(Nc).astype(np.uint64)
    tableau=np.append(a,I,axis=1)+Fraction()
    tableau=np.vstack([tableau.T,b])+Fraction()
    tableau=tableau.T
    tableau=np.vstack([tableau,c])+Fraction()
    tableau=tableau+Fraction()
    return tableau

In [10]:
def simplex(a):
    fv=[]
    infeasible=0
    Nc=np.shape(A)[0]
    Nv=np.shape(A)[1]
    for i in range(Nc):
        fv.append(Nv+i)
    for k in range(1001):
        if(np.all(a[-1,0:-1]>=0) and np.all(a[0:-1,-1]>=0)):
            a=a+Fraction()
            return a,fv,infeasible
        else :
            column=np.argmin(a[-1,0:-1]) 
            if a[-1][column]<0:
                b=(a[0:-1,column])
                b=np.where(b==0,1/10000000,b)
                ratio=((a[0:-1,-1])/b)+Fraction()
                ratio=np.where(ratio<0,100000,ratio)
                for pivot in range(len(ratio)):
                    if ratio[pivot]==0 and b[pivot]<=0 :
                        ratio[pivot]=10000
                row=np.argmin(ratio)
                a[row,:]=a[row,:]/a[row][column]+Fraction()
                fv[row]=column
                for i in range(Nc+1):
                    if(i!=row):
                        a[i,:]=a[i,:]-a[i][column]*a[row,:]
                continue
            else :
                for i in range(Nc):  
                    if(a[i][-1]<0):
                        column=np.argmin(a[i,:-1])
                        if a[i][column]<0:
                            row=i
#                             b=(a[0:-1,column])
#                             b=np.where(b==0,1/1000000000,b)
#                             ratio=((a[0:-1,-1])/b)
#                             ratio=np.where(ratio<0,10000,ratio)
#                             for pivot in range(len(ratio)):
#                                 if ratio[pivot]>=0 and ratio[pivot]<row and b[pivot]>0 :
#                                     row=pivot
                            a[row,:]=a[row,:]/a[row][column]+Fraction()
                            fv[row]=column
                            for j in range(Nc+1):
                                if(j!=row):
                                    a[j,:]=a[j,:]-a[j][column]*a[row,:]+Fraction()
                            i=0
                        else :
                            infeasible=1
                            a=a+Fraction()
                            return a,fv,infeasible
    a=a+Fraction()
    return a,fv,infeasible
            

In [11]:
def onecut(A,B,C):
    Nv=np.shape(A)[1] # no of variables
    flag=0
    Nc=np.shape(A)[0]
    D=np.zeros([Nv,Nv+1])
    for i in range(Nv):
        D[i,i]=-1
    E=np.vstack((A.T,B))
    E=E.T
    D=np.vstack((D,E,np.zeros(Nv+1)))
    D[-1,-1] = -1 
    D=D.astype(np.int64)+Fraction()
    tableau=preprocessing_CP(C,A,B)
    sol_tab,solutions,infeasible=simplex(tableau)
    print("\n",sol_tab,solutions)   # This function is used for printing simplex tableau after solving 
    z1=np.floor(sol_tab).astype(np.int64)+Fraction()
    z=(sol_tab-z1)+Fraction()
    x=np.zeros(Nv)
    for j in range(Nv) :
        for i in range(len(solutions)):
            if solutions[i]==j:
                x[j]=sol_tab[i,-1]
                
    new_const=np.zeros(Nv+1).astype(np.int64)+Fraction()
    
    if(infeasible==1) or np.any(sol_tab[-1,0:-1]<-0.001) or np.any(sol_tab[0:-1,-1]<-0.001):
        flag=100
        return A,B,x,sol_tab[-1][-1],flag,new_const
        
    if np.all(z[0:-1,-1]<=0.001) :
        flag=1
    
    else :
        k=0
        for i in range(np.shape(z)[0]-1):
            if(z[i,-1]>=0.001):
                k=i
                for j in range(0,Nv+Nc+1):
                    new_const+=(z[k][j]*D[j,:])  
                new_const=new_const.astype(np.int64)
                
                A=np.vstack([A,new_const[:-1]])
                B=np.hstack((B,new_const[-1]))
                dupli=np.vstack((A.T,B))
                dupli=(dupli.T).astype(np.int64)
                dupli2=(np.unique(dupli,axis=0))
                if(np.shape(dupli)[0]==np.shape(dupli2)[0]) and np.all(new_const-np.floor(new_const)==0):
                    break
                else :
                    A=A[:-1,:]
                    B=B[:-1]
        if(i==np.shape(z)[0]-2):
            flag=100
            A=A.astype(np.int64)+Fraction()
            B=B.astype(np.int64)+Fraction()
            return A,B,x,sol_tab[-1][-1],flag,new_const
            
    A=A.astype(np.int64)+Fraction()
    B=B.astype(np.int64)+Fraction()   
    return A,B,x,sol_tab[-1][-1],flag,new_const

This block will run the cutting planes algorithm till we get the solution, or we get some error , either solution is infeasible
or 0/0 fraction term comes while solving simplex tableau

In [12]:
start_time=time.time()
for k in range(100):
    A,B,x,sol,flag,new_const=onecut(A,B,C)
    if flag==1:
        print("Solution complete ",x,sol)
        break
    elif flag==0: 
        print("\nSolution incomplete \nConstraint :",new_const)
    else:
        print("Constraint not found or solution is infeasible or unbounded",x,sol)
        break
print("Time taken ",np.round(time.time()-start_time,3),"\n No. of iterations :",k+1)


 [[Fraction(0, 1) Fraction(0, 1) Fraction(1, 1) Fraction(2, 3)
  Fraction(0, 1) Fraction(0, 1) Fraction(-1, 3) Fraction(0, 1)
  Fraction(0, 1) Fraction(16, 3)]
 [Fraction(1, 1) Fraction(3, 5) Fraction(0, 1) Fraction(-1, 10)
  Fraction(1, 100) Fraction(0, 1) Fraction(2, 5) Fraction(0, 1)
  Fraction(0, 1) Fraction(7, 2)]
 [Fraction(0, 1) Fraction(1, 5) Fraction(0, 1) Fraction(-1, 5)
  Fraction(1, 50) Fraction(1, 1) Fraction(4, 5) Fraction(0, 1)
  Fraction(0, 1) Fraction(0, 1)]
 [Fraction(0, 1) Fraction(2, 5) Fraction(0, 1) Fraction(1, 10)
  Fraction(-1, 100) Fraction(0, 1) Fraction(-2, 5) Fraction(1, 1)
  Fraction(0, 1) Fraction(13, 2)]
 [Fraction(0, 1) Fraction(0, 1) Fraction(0, 1) Fraction(1, 3)
  Fraction(0, 1) Fraction(0, 1) Fraction(1, 3) Fraction(0, 1)
  Fraction(1, 1) Fraction(2, 3)]
 [Fraction(0, 1) Fraction(4, 1) Fraction(0, 1) Fraction(6, 1)
  Fraction(2, 5) Fraction(0, 1) Fraction(6, 1) Fraction(0, 1)
  Fraction(0, 1) Fraction(300, 1)]] [2, 0, 5, 7, 8]

Solution incomplete 
C